<a href="https://colab.research.google.com/github/sant-sh/pdf/blob/main/PDF_to_QA_Dataset_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

In [ ]:
!pip install -qU langchain langchain-community tiktoken
!pip install pypdf

In [ ]:
!nvidia-smi


Sun May 26 13:22:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes

os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [ ]:
torch.set_default_device('cuda')

In [ ]:
# use the commented out parts for running in 4bit
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3",
                                             quantization_config=quantization_config,
                                            #  low_cpu_mem_usage=True,
                                            #  torch_dtype="auto",
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

32768

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.0
# generation_config.top_p = 0.7
# generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config


GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.0
}

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",
)

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:

def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt +'\n\n' + input_text},
    ]

    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
            )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=max_length,
        eos_token_id=terminators,
        pad_token_id=pipe.tokenizer.eos_token_id,
        do_sample=False,
        # temperature=0.0,
        # top_p=0.9,
    )

    generated_outputs = outputs[0]["generated_text"]
    text = outputs[0]["generated_text"][len(prompt):]
    wrapped_text = wrap_text(text)
    return wrapped_text

In [ ]:
# generate('Write a detailed analogy between mathematics and a lighthouse.',
#          system_prompt="You are a helpful assistant called Mistral. Write out your reasoning step-by-step to be sure you get the right answers!",
#          max_length=1024)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Analogies help us understand complex concepts by comparing them to something more
familiar. In this case, I will compare mathematics to a lighthouse, focusing on the
aspects of guidance, structure, and continuous learning.

1. Guidance:
   - Mathematics: Provides a systematic and logical framework for solving problems and
understanding the world around us.
   - Lighthouse: Guides sailors safely to shore by emitting a beam of light, helping them
navigate through potentially dangerous waters.

2. Structure:
   - Mathematics: Consists of axioms, theorems, and proofs, which form a hierarchical
structure of knowledge.
   - Lighthouse: Has a tower and a beacon, which are the physical structure that supports
the light emission.

3. Continuous Learning:
   - Mathematics: Is a lifelong learning process, with new concepts, theorems, and
applications constantly being discovered and developed.
   - Lighthouse: Requires regular maintenance and upgrades to ensure the light remains
strong and visibl

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Insert how many pdfs you want
loaders = [
 PyPDFLoader("Electrical_Installation_Guide_2018.pdf"),
]

data = []
for loader in loaders:
    data.extend(loader.load())

In [ ]:
# Transform (Split to chunks)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20,
                                               separators=["\n\n", "\n", "(?<=\. )", " "], length_function=len)
docs = text_splitter.split_documents(data)
print('Split into ' + str(len(docs)) + ' docs')

Split into 1951 docs


In [ ]:
docs_text = [doc.page_content for doc in docs]

In [ ]:
def get_custom_prompt(input_chunk):
  custom_prompt = '''
  Instructions:
  1. For each chunk of text inputed, generate one relevant question and its answer.
  2. Keep that that one question concise and contextually appropriate.
  3. Ensure that that one answer are informative and accurate.
  4. Use the provided input as a basis for generating questions.

  Generate ONE question and its corresponding answer based on the entire chunk context in the last example. Follow the format of the examples below to include context, question, and answer in the response

  Few-Shot Prompt (Context):
  ---
  {
    "context": "In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
    "question": "Who published A Mathematical Theory of Communication in 1948?",
    "answer": "Claude E. Shannon."
  }
  ---

  Now proceed with generating questions and answers for the remaining text chunks.
  output:
  {
    "context": "{input_chunk}",
    "question": "[The generated question from the chunk]",
    "answer": "[the answer to the question]"
  }
  '''
  return custom_prompt

In [ ]:
print(docs_text[164])

v The communication with the remote control and monitoring center
b DC auxiliary supply in every substation.
As described above, most of the loops are historically equipped with load break 
switches and protected at each end only by circuit breakers located in the 
HV/ MV primary substations. In case of fault, all the customers supplied by a faulty 
feeder are disconnected. But in fact the customers upstream from the fault could 
have not been disconnected.
The addition of circuit breakers, adequately located and associated with 
appropriate protection relays may reduce the number of customers disconnected in 
case of fault.
As an example, a loop including two additional circuit breakers is divided in 
four independent sections. Assume the open point located between the two 
additional circuit breakers. In case of fault in the section delimited by these two 
circuit breakers only a part of the secondary substations of the section will be


In [ ]:

json_format = generate(docs_text[164],
                system_prompt=get_custom_prompt(docs_text[164]),
                max_length=1024)
print(json_format)

 {
  "context": "The addition of circuit breakers, adequately located and associated with
appropriate protection relays may reduce the number of customers disconnected in case of
fault. As an example, a loop including two additional circuit breakers is divided in four
independent sections. Assume the open point located between the two additional circuit
breakers.",
  "question": "What is an example of a power system configuration that may reduce the
number of customers disconnected in case of fault?",
  "answer": "A power system configuration that includes a loop with two additional circuit
breakers, divided into four independent sections, is an example."
}


In [ ]:
import json

# Your provided JSON string
def convert_string_to_json(json_format):
  try:
    new_string = json_format
    # Read existing data from the JSON file (if it exists)
    try:
        with open("dataset.json", "r") as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        # If the file doesn't exist yet, create an empty list
        existing_data = []

    # Append the new string to the existing data
    # new_string = json.dumps(json_format)
    new_string = new_string.replace('\r\n', ' ')
    new_string = new_string.replace('\r\\n', ' ')
    new_string = new_string.replace('\\"', '"')
    new_string = json.loads(new_string.strip(), strict=False)




    existing_data.append(new_string)
    # Write back to the JSON file
    with open("dataset.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)

    print("New string added to the JSON file successfully!")
  except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
for index, chunk in enumerate(docs_text):
  if index % 100 == 0:
    print(f"Epoch: {index}/{len(docs_text)}")

  # Colab free version can do ~100 epoch per hour
  if index > 300: # continue from where it stopped
    json_format = generate(chunk,
                  system_prompt=get_custom_prompt(chunk),
                  max_length=1024)
    convert_string_to_json(json_format)

print("Training complete")
